<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/Points_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#------------------------------------------------------
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#  MAKE SURE TO CHANGE GAMEWEEK TO THE GW YOU WANT TO PREDICT
# SAVE TO GITHUB WHEN FINISHED

In [2]:
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards",
    "team Goal scored",
    "team Goal conceded"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3','mean assists all',
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


date_cols=["day_of week","month","hour","week"]

In [3]:
def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0

In [4]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.9 MB/s eta 0:00:00


In [5]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

In [6]:
import pandas as pd
import numpy as np
import warnings
import os


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold

In [7]:
# ************ CHANGE TO NEXT GW TO BE PREDICTED ******************************
gameweek=15
# *****************************************************************************

In [8]:
# RELOAD THIS DATA FOR EACH MODEL

train = pd.read_csv("/content/gdrive/MyDrive/fplpredict/cleaned_dataset/cleaned_previous_seasons.csv",index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
old_gameweek_cleaned = []
for i in range(1, gameweek):
    old_gameweek_cleaned.append(pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{i}.csv"))
old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{gameweek}.csv", index_col=0)

In [9]:
#CREATE GAMEWEEK PREDICTIONS FOLDER
path = f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

# data from current season but previous gameweeks

Folder /content/gdrive/MyDrive/fplpredict/predicted_dataset/GW15 already exists


In [10]:
train["position"].value_counts()

,count
position,
MID,53951
DEF,43224
FWD,16079
GK,13166
GKP,1097


In [11]:
train["position"]=train["position"].replace({"GKP":"GK" })

In [12]:
#Predict Points - MAKE SURE TO READ THE DATA IN AGAIN

train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
#train["date"]=pd.to_datetime(train["kickoff_time"])
train["date"] = pd.to_datetime(train["kickoff_time"], format='ISO8601', errors='coerce')
#train["day_of week"]=train["date"].dt.day_name
train["day_of week"] = train["date"].dt.day_name()  # Added parentheses to call the function
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
#train["week"]=train["date"].dt.week
train["week"] = train["date"].dt.isocalendar().week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
#test["date"]=pd.to_datetime(test["kickoff_time"])
test["date"] = pd.to_datetime(test["kickoff_time"], format='ISO8601', errors='coerce')
#test["day_of week"]=test["date"].dt.day_name
test["day_of week"] = test["date"].dt.day_name()  # Added parentheses to call the function
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
#test["week"]=test["date"].dt.week
test["week"] = test["date"].dt.isocalendar().week
test.drop(["kickoff_time","date"], axis=1, inplace=True)

target = train[["total_points", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["position"]=train["position"].astype("category")
test["position"]=test["position"].astype("category")
train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns+date_cols]
test= test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["total_points"],
    test_size=0.1,
    random_state=0,
)

In [13]:
from sklearn.model_selection import KFold

#cross_validator to split the data into folds
folds=KFold(n_splits=8,shuffle=True,random_state=0)

#a dataframe to store the predictions made by each fold
predictions_df=pd.DataFrame()

#list to save the mean absolute errors from validating on each folds
rmse_val=[]
rmse_X=[]

#a simple catboost regressor
model=LGBMRegressor(**{'colsample_bytree': 0.4199299182268318, 'learning_rate': 0.0032874466037521254, 'max_depth': 9, 'min_split_gain': 0.5685369160138952, 'num_leaves': 99, 'reg_alpha': 0.5621526419488447, 'reg_lambda': 0, 'subsample': 0.6534153111773866}, verbose=-50,random_state=0,early_stopping_rounds=200,n_estimators=10000)

#train model, make predictions and check the validation accuracy on  each fold
for i,(train_index,test_index) in enumerate(folds.split(train.drop(leak_columns, axis=1),target["total_points"])):
    train_fold=train.drop(leak_columns, axis=1).iloc[train_index]
    val_fold=train.drop(leak_columns, axis=1).iloc[test_index]
    y_fold=target["total_points"].iloc[train_index]
    y_val_fold=target["total_points"].iloc[test_index]


    model.fit(train_fold,y_fold,eval_set=[(val_fold,y_val_fold)])
    print(i+1)
    prediction=model.predict(test.drop(leak_columns, axis=1))
    predictions_df[i]=prediction
    rmse_val.append(mean_squared_error(model.predict(val_fold),y_val_fold,squared=False))
    rmse_X.append(mean_squared_error(model.predict(train_fold),y_fold,squared=False))
print(rmse_val)
print(rmse_X)

1
2
3
4
5
6
7
8
[1.9807687365586903, 1.9676063152796164, 1.9956498833258425, 1.972092684336828, 2.021609160316208, 1.9493837099079843, 1.9696809382747165, 1.9645442223476766]
[1.4460408071641377, 1.6395878866948437, 1.7340971154922247, 1.5047116773877562, 1.4276055529676457, 1.3369846769245806, 1.4976038694919875, 1.6409450397662724]


In [14]:
print(np.mean(rmse_val))
print(np.mean(rmse_X))
#2.7575293874474336   1.9742746163696216  1.9726217007233091
#2.0174134721607295   1.564712957980843   1.5033778434088017

1.9776669562934452
1.528447078236181


In [15]:
predictions_df[0]

,0
0,0.032191
1,0.943926
2,0.058090
3,2.705862
4,0.100794
...,...
688,0.121325
689,0.031239
690,0.233229
691,0.105944


In [16]:
test["points"]=np.mean(predictions_df, axis=1).values

test[leak_columns + ["points", "value"]].sort_values(
    "points", ascending=False
).to_csv("points.csv")

In [17]:
test[test["position"]=="MID"].sort_values(by="points",ascending=False).head(11)[["name","points","team"]]

,name,points,team
index,,,
Mohamed Salah2024-12-07T12:30:00Z,Mohamed Salah,9.355927,Liverpool
Bukayo Saka2024-12-08T14:00:00Z,Bukayo Saka,6.563140,Arsenal
Jarrod Bowen2024-12-09T20:00:00Z,Jarrod Bowen,5.494858,West Ham
Cole Palmer2024-12-08T16:30:00Z,Cole Palmer,5.269450,Chelsea
Bryan Mbeumo2024-12-07T15:00:00Z,Bryan Mbeumo,4.773502,Brentford
Bruno Borges Fernandes2024-12-07T17:30:00Z,Bruno Borges Fernandes,4.771218,Man Utd
Martin Ødegaard2024-12-08T14:00:00Z,Martin Ødegaard,4.734985,Arsenal
Kevin De Bruyne2024-12-07T15:00:00Z,Kevin De Bruyne,4.511795,Man City
James Maddison2024-12-08T16:30:00Z,James Maddison,4.387995,Spurs


In [18]:
test[test["position"]=="DEF"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Aaron Wan-Bissaka2024-12-09T20:00:00Z,Aaron Wan-Bissaka,4.180975,West Ham
Virgil van Dijk2024-12-07T12:30:00Z,Virgil van Dijk,4.134795,Liverpool
Andrew Robertson2024-12-07T12:30:00Z,Andrew Robertson,3.737932,Liverpool
Max Kilman2024-12-09T20:00:00Z,Max Kilman,3.636714,West Ham
William Saliba2024-12-08T14:00:00Z,William Saliba,3.607348,Arsenal
Lucas Digne2024-12-07T15:00:00Z,Lucas Digne,3.472141,Aston Villa
Trent Alexander-Arnold2024-12-07T12:30:00Z,Trent Alexander-Arnold,3.348457,Liverpool
Vitalii Mykolenko2024-12-07T12:30:00Z,Vitalii Mykolenko,3.255345,Everton
Ethan Pinnock2024-12-07T15:00:00Z,Ethan Pinnock,3.223703,Brentford


In [19]:
test[test["position"]=="GKP"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
André Onana2024-12-07T17:30:00Z,André Onana,3.924138,Man Utd
Caoimhin Kelleher2024-12-07T12:30:00Z,Caoimhin Kelleher,3.560543,Liverpool
Łukasz Fabiański2024-12-09T20:00:00Z,Łukasz Fabiański,3.365212,West Ham
David Raya Martin2024-12-08T14:00:00Z,David Raya Martin,3.362971,Arsenal
Mark Flekken2024-12-07T15:00:00Z,Mark Flekken,3.243706,Brentford
Bernd Leno2024-12-08T14:00:00Z,Bernd Leno,3.111523,Fulham
Jordan Pickford2024-12-07T12:30:00Z,Jordan Pickford,3.049480,Everton
Kepa Arrizabalaga2024-12-08T14:00:00Z,Kepa Arrizabalaga,2.881691,Bournemouth
Bart Verbruggen2024-12-08T14:00:00Z,Bart Verbruggen,2.705136,Brighton


In [20]:
test[test["position"]=="FWD"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Erling Haaland2024-12-07T15:00:00Z,Erling Haaland,5.153509,Man City
Ollie Watkins2024-12-07T15:00:00Z,Ollie Watkins,4.476092,Aston Villa
Chris Wood2024-12-07T17:30:00Z,Chris Wood,4.352171,Nott'm Forest
Alexander Isak2024-12-07T15:00:00Z,Alexander Isak,3.955067,Newcastle
Matheus Santos Carneiro Da Cunha2024-12-09T20:00:00Z,Matheus Santos Carneiro Da Cunha,3.934838,Wolves
Nicolas Jackson2024-12-08T16:30:00Z,Nicolas Jackson,3.826613,Chelsea
Yoane Wissa2024-12-07T15:00:00Z,Yoane Wissa,3.725058,Brentford
João Pedro Junqueira de Jesus2024-12-08T14:00:00Z,João Pedro Junqueira de Jesus,3.541342,Brighton
Liam Delap2024-12-08T14:00:00Z,Liam Delap,3.349525,Ipswich


In [21]:
test["points"].sort_values(ascending=False).head(50)

,points
index,
Mohamed Salah2024-12-07T12:30:00Z,9.355927
Bukayo Saka2024-12-08T14:00:00Z,6.563140
Jarrod Bowen2024-12-09T20:00:00Z,5.494858
Cole Palmer2024-12-08T16:30:00Z,5.269450
Erling Haaland2024-12-07T15:00:00Z,5.153509
Bryan Mbeumo2024-12-07T15:00:00Z,4.773502
Bruno Borges Fernandes2024-12-07T17:30:00Z,4.771218
Martin Ødegaard2024-12-08T14:00:00Z,4.734985
Kevin De Bruyne2024-12-07T15:00:00Z,4.511795


In [22]:
feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)

In [ ]:
feature_importance.head(50)

,column,imp
71,opp mean team Goal scored all,5970
72,opp mean team Goal conceded all,5842
73,opp mean match_result all,5521
29,mean ict_index 3,4465
24,mean bps 3,4390
66,mean team Goal conceded all,4376
64,mean team Goal scored all,4304
4,percent_value,4238
62,mean match_result all,4159
26,mean creativity 3,3988


In [23]:
feature_importance.tail(30)

,column,imp
13,threat_ex,4214
63,mean team Goal scored 3,4122
16,ict_index_ex,4080
12,influence_ex,3957
76,hour,3814
3,last_season_position,3687
27,mean goals_conceded 3,3395
17,clean_sheets_ex,2927
14,bonus_ex,2892
65,mean team Goal conceded 3,2841


In [ ]:
# Save predictions

In [24]:
test[test["position"]=="DEF"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defenders_points.csv")
test[test["position"]=="GKP"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_points.csv")
test[test["position"]=="MID"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_points.csv")
test[test["position"]=="FWD"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_points.csv")